<a href="https://colab.research.google.com/github/SarahYLawrence/Text-Mining-and-Analytics/blob/Task-3/Erisk_T3_dataclear2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import os

def merge_text_from_csv(csv_folder, txt_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(txt_folder):
        os.makedirs(txt_folder)

    # Loop through each file in the input folder
    for file_name in os.listdir(csv_folder):
        if file_name.endswith('.csv'):
            csv_file = os.path.join(csv_folder, file_name)
            output_file = os.path.join(txt_folder, os.path.splitext(file_name)[0] + '.txt')
            merge_text_from_single_csv(csv_file, output_file)

def merge_text_from_single_csv(csv_file, output_file):
    with open(csv_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        text_data = []
        for row in reader:
            text_data.append(row['Text'])

    with open(output_file, 'w', encoding='utf-8') as txtfile:
        txtfile.write('\n'.join(text_data))

# Replace 'input_folder' with the path to the folder containing CSV files
# Replace 'output_folder' with the path to the folder where you want to save the text files
merge_text_from_csv('2022', '2022txt')
merge_text_from_csv('2023', '2023txt')


In [ ]:
!zip -r 2022txt.zip 2022txt/
!zip -r 2023txt.zip 2023txt/


In [ ]:
import csv
import os
import json

def merge_text_and_create_json(csv_folder, txt_folder, json_file):
    data = []
    # Create the output folder if it doesn't exist
    if not os.path.exists(txt_folder):
        os.makedirs(txt_folder)

    # Loop through each file in the input folder
    for file_name in os.listdir(csv_folder):
        if file_name.endswith('.csv'):
            csv_file = os.path.join(csv_folder, file_name)
            output_file = os.path.join(txt_folder, os.path.splitext(file_name)[0] + '.txt')
            text_data = merge_text_from_single_csv(csv_file, output_file)
            data.append({"file_name": file_name, "text": text_data})

    # Write the data to the JSON file
    with open(json_file, 'w', encoding='utf-8') as jsonfile:
        json.dump(data, jsonfile, indent=4)

def merge_text_from_single_csv(csv_file, output_file):
    with open(csv_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        text_data = []
        for row in reader:
            text_data.append(row['Text'])

    with open(output_file, 'w', encoding='utf-8') as txtfile:
        txtfile.write('\n'.join(text_data))

    return text_data

# Replace 'input_folder' with the path to the folder containing CSV files
# Replace 'output_folder' with the path to the folder where you want to save the text files
# Replace 'output_json.json' with the desired name for the JSON file
merge_text_and_create_json('2022', '2022txt', 'output_json2022.json')
merge_text_and_create_json('2023', '2023txt', 'output_json2023.json')


In [ ]:
!pip install sumy


In [ ]:
import csv
import os
import json
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk
nltk.download('punkt')

def merge_text_and_create_json(csv_folder, txt_folder, json_file):
    data = []
    # Create the output folder if it doesn't exist
    if not os.path.exists(txt_folder):
        os.makedirs(txt_folder)

    # Loop through each file in the input folder
    for file_name in os.listdir(csv_folder):
        if file_name.endswith('.csv'):
            csv_file = os.path.join(csv_folder, file_name)
            output_file = os.path.join(txt_folder, os.path.splitext(file_name)[0] + '.txt')
            text_data = merge_text_from_single_csv(csv_file, output_file)

            # Summarize the text data
            summary = summarize_text('\n'.join(text_data))

            # Append the summarized text along with the original text to the data list
            data.append({"file_name": file_name, "text": text_data, "summary": summary})

    # Write the data to the JSON file
    with open(json_file, 'w', encoding='utf-8') as jsonfile:
        json.dump(data, jsonfile, indent=4)

def merge_text_from_single_csv(csv_file, output_file):
    with open(csv_file, 'r', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        text_data = []
        for row in reader:
            text_data.append(row['Text'])

    with open(output_file, 'w', encoding='utf-8') as txtfile:
        txtfile.write('\n'.join(text_data))

    return text_data

def summarize_text(text):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, 2)  # Summarize to 2 sentences
    return " ".join([str(sentence) for sentence in summary])

# Replace 'input_folder' with the path to the folder containing CSV files
# Replace 'output_folder' with the path to the folder where you want to save the text files
# Replace 'output_json.json' with the desired name for the JSON file
merge_text_and_create_json('2022', '2022txt', 'output_json2022.json')
merge_text_and_create_json('2023', '2023txt', 'output_json2023.json')


In [ ]:
!pip install sentence-transformers numpy


In [ ]:
import json
import os
import numpy as np
from sentence_transformers import SentenceTransformer

def generate_vectors_from_json(json_file):
    # Load SBERT model
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    # Load data from JSON file
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Generate vectors for each text
    for entry in data:
        text = entry['text']
        vectors = generate_vectors(text, model)
        entry['vector'] = vectors

    # Update JSON file with vectors
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4, default=lambda x: x.tolist())

def generate_vectors(texts, model):
    return [model.encode(text).tolist() for text in texts]  # Convert numpy array to list

# Replace 'json_file_path' with the path to your JSON file
json_file_path = 'output_json2022.json'

# Generate vectors and update JSON file
generate_vectors_from_json(json_file_path)
